In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.models as models
import datetime
from random import shuffle

In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
kwargs = {'num_workers': 1, 'pin_memory': True}
device = torch.device("cuda")
batch_size = 64
test_batch_size = 1000
image_size = (32,32)
seed = 10
torch.manual_seed(seed)

# Loading Train CIFAR 10 dataset

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified


# Loading Test CIFAR 10 dataset

In [4]:
# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified


# Neural Network Structure

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)



# Training Function

In [6]:
def train(epoch, device, trainloader):
    model.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = F.nll_loss(outputs, labels)
        loss.backward()
        optimizer.step()

# Test Function

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Run everyting

In [8]:
model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, 51):
    print(datetime.datetime.now())
    print("epoch: " + str(epoch))
    train(epoch, device, trainloader)
    test(model, device, testloader)
#save model
torch.save(model.state_dict(), ("cifar_model.pt"))



2019-06-18 16:51:20.945481
epoch: 1
Test set: Average loss: 1.9809, Accuracy: 2837/10000 (28%)

2019-06-18 16:51:34.390148
epoch: 2
Test set: Average loss: 1.7325, Accuracy: 3665/10000 (37%)

2019-06-18 16:51:46.978085
epoch: 3
Test set: Average loss: 1.6315, Accuracy: 3970/10000 (40%)

2019-06-18 16:51:59.787958
epoch: 4
Test set: Average loss: 1.8664, Accuracy: 2966/10000 (30%)

2019-06-18 16:52:12.272298
epoch: 5
Test set: Average loss: 1.4561, Accuracy: 4684/10000 (47%)

2019-06-18 16:52:24.469368
epoch: 6
Test set: Average loss: 1.5562, Accuracy: 4454/10000 (45%)

2019-06-18 16:52:36.694437
epoch: 7
Test set: Average loss: 1.4192, Accuracy: 4864/10000 (49%)

2019-06-18 16:52:48.854515
epoch: 8
Test set: Average loss: 1.4128, Accuracy: 4944/10000 (49%)

2019-06-18 16:53:00.989606
epoch: 9
Test set: Average loss: 1.4913, Accuracy: 4754/10000 (48%)

2019-06-18 16:53:13.165682
epoch: 10
Test set: Average loss: 1.4189, Accuracy: 5014/10000 (50%)

2019-06-18 16:53:25.394741
epoch: 11
Te